In [1]:
%load_ext autoreload
%autoreload 2

starting from the part 1 solution code, replacing the full input with the very simple example

In [2]:
from part_01_solution import (
    positions,
    crossed_paths,
    ORIGIN
)

[['R8', 'U5', 'L5', 'D3'], ['U7', 'R6', 'D4', 'L4']]

There were 12 times the paths of the wires crossed.
The min. distance from one and the point of origin was 1.



It seems like something is off here already...these wires should only cross twice, not 12 times.  And the minimum distance between a crossing and the origin is 6 away

Going to restart the kernel again just in case

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from part_01_solution import (
    positions,
    crossed_paths,
    ORIGIN
)

[['R8', 'U5', 'L5', 'D3'], ['U7', 'R6', 'D4', 'L4']]

There were 12 times the paths of the wires crossed.
The min. distance from one and the point of origin was 1.



In [3]:
positions

{'wire_a': [(0, 0),
  (1, 0),
  (2, 0),
  (3, 0),
  (4, 0),
  (5, 0),
  (6, 0),
  (7, 0),
  (8, 0),
  (8, 1),
  (8, 2),
  (8, 3),
  (8, 4),
  (8, 5),
  (7, 5),
  (6, 5),
  (5, 5),
  (4, 5),
  (3, 5),
  (3, 4),
  (3, 3),
  (3, 2)],
 'wire_b': [(0, 0),
  (1, 0),
  (2, 0),
  (3, 0),
  (4, 0),
  (5, 0),
  (6, 0),
  (7, 0),
  (7, 1),
  (7, 2),
  (7, 3),
  (7, 4),
  (7, 5),
  (7, 6),
  (6, 6),
  (5, 6),
  (4, 6),
  (3, 6),
  (3, 5),
  (3, 4),
  (3, 3),
  (3, 2)]}

Those look fine

Here's the code that is calculating the crossed paths

In [4]:
crossed_paths = list(set(positions["wire_a"]).intersection(positions["wire_b"]))

In [5]:
crossed_paths

[(3, 5),
 (0, 0),
 (7, 0),
 (3, 3),
 (3, 2),
 (3, 0),
 (6, 0),
 (2, 0),
 (7, 5),
 (5, 0),
 (1, 0),
 (3, 4),
 (4, 0)]

Again, too high.  There should only be 2

In [6]:
set(positions["wire_a"])

{(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (4, 0),
 (4, 5),
 (5, 0),
 (5, 5),
 (6, 0),
 (6, 5),
 (7, 0),
 (7, 5),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5)}

In [7]:
len(set(positions["wire_a"]))

22

In [8]:
len(positions["wire_a"])

22

I don't know why the `set` invocation is there, since each position is unique

Oh, probably to make the `intersection` method available.  But let's confirm that it's doing what we want

In [9]:
set(positions["wire_a"]).intersection(positions["wire_b"])

{(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (7, 5)}

Wait actually I don't think this is the problem.  (1, 0) appears in both, but it _shouldn't_.  The second wire goes up 7 times, never should pass through (1, 0)

The logic making the `positions` variable seems to be assuming that the wire always moves right, up, left, down, but this is only true of the first wire, not the second wire

In [10]:
from part_01_solution import calculate_relative_coords

In [11]:
calculate_relative_coords(["U7","R6","D4","L4"])

[(7, 6), (-4, -4)]

Yeah this is assuming that the first direction is always right or left, second is up or down, third is right or left, fourth is up or down.  But this is not a correct assumption

I think I can rewrite just this function, to make four vectors for four wire instructions instead of two vectors for four wire instructions.

In [13]:
from typing import List, Tuple

# create new data type
CoordPair = List[Tuple[int, int]]

def calculate_relative_coords(directions: List[str]) -> CoordPair:
    """Determines the relative coordinate pair for each step along a wire
    Note:
        - The key term here is 'relative'. Each coordinate pair is a step in
          both the x and y direction relative to the previous step. The pairs
          are meaningless by themselves.
    Args:
        - direction (List[str]): the direction recorded at a particular point
                                 along a wire
    Results:
        List[Tuple[int, int]]: List of each coordinate pair, stored as a tuple
    """
    coords = []
    
    for instruction in directions:
        direction = instruction[0]
        magnitude = int(instruction[1:])
        
        if direction == "R":
            coords.append((magnitude, 0))
        elif direction == "L":
            coords.append((-magnitude, 0))
        elif direction == "U":
            coords.append((0, magnitude))
        elif direction == "D":
            coords.append((0, -magnitude))
        else:
            print("Error, encountered direction", direction)
            
    return coords

In [14]:
calculate_relative_coords(["U7","R6","D4","L4"])

[(0, 7), (6, 0), (0, -4), (-4, 0)]

In [15]:
example_input = """R8,U5,L5,D3
U7,R6,D4,L4"""
wires = [wire.split(",") for wire in example_input.split("\n")]

In [16]:
relative_positions = {"wire_a": calculate_relative_coords(wires[0]),
                      "wire_b": calculate_relative_coords(wires[1])}

In [18]:
from part_01_solution import calculate_positions
positions = {"wire_a": calculate_positions(relative_positions["wire_a"]),
             "wire_b": calculate_positions(relative_positions["wire_b"])}

In [19]:
positions

{'wire_a': [(0, 0),
  (1, 0),
  (2, 0),
  (3, 0),
  (4, 0),
  (5, 0),
  (6, 0),
  (7, 0),
  (8, 0),
  (8, 1),
  (8, 2),
  (8, 3),
  (8, 4),
  (8, 5),
  (7, 5),
  (6, 5),
  (5, 5),
  (4, 5),
  (3, 5),
  (3, 4),
  (3, 3),
  (3, 2)],
 'wire_b': [(0, 0),
  (0, 1),
  (0, 2),
  (0, 3),
  (0, 4),
  (0, 5),
  (0, 6),
  (0, 7),
  (1, 7),
  (2, 7),
  (3, 7),
  (4, 7),
  (5, 7),
  (6, 7),
  (6, 6),
  (6, 5),
  (6, 4),
  (6, 3),
  (5, 3),
  (4, 3),
  (3, 3),
  (2, 3)]}

In [20]:
crossed_paths = list(set(positions["wire_a"]).intersection(positions["wire_b"]))

In [21]:
crossed_paths

[(0, 0), (3, 3), (6, 5)]

In [22]:
crossed_paths.remove(ORIGIN)

In [24]:
from part_01_solution import distance

distances = []
for path in crossed_paths:
    distances.append(distance(ORIGIN, path, m=1))

print(f"""
There were {len(crossed_paths)} times the paths of the wires crossed.
The min. distance from one and the point of origin was {round(min(distances))}.
""")


There were 2 times the paths of the wires crossed.
The min. distance from one and the point of origin was 6.



Ok, that is correct behavior for part 1, trying code from part 2

In [25]:
# for each crossed path, sum the steps it took both wires to arrive there
steps = {"path": [], "steps": []}
for path in crossed_paths:
    steps["path"].append(path)
    steps["steps"].append(positions["wire_a"].index(path)
                          + positions["wire_b"].index(path))

# find the minimum number of steps it took to reach one intersection
print(f"""
The min. number of steps to reach an intersection is {min(steps["steps"])}.
""")


The min. number of steps to reach an intersection is 30.



That is correct, let's try the more complicated ones

In [26]:
example_input = """R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83"""
wires = [wire.split(",") for wire in example_input.split("\n")]

relative_positions = {"wire_a": calculate_relative_coords(wires[0]),
                      "wire_b": calculate_relative_coords(wires[1])}

positions = {"wire_a": calculate_positions(relative_positions["wire_a"]),
             "wire_b": calculate_positions(relative_positions["wire_b"])}

crossed_paths = list(set(positions["wire_a"]).intersection(positions["wire_b"]))

crossed_paths.remove(ORIGIN)

# remove the origin from both wires
for wire in positions:
    positions[wire].remove(ORIGIN)

In [27]:
# for each crossed path, sum the steps it took both wires to arrive there
steps = {"path": [], "steps": []}
for path in crossed_paths:
    steps["path"].append(path)
    steps["steps"].append(positions["wire_a"].index(path)
                          + positions["wire_b"].index(path))

# find the minimum number of steps it took to reach one intersection
print(f"""
The min. number of steps to reach an intersection is {min(steps["steps"])}.
""")


The min. number of steps to reach an intersection is 608.



That's off by 2 when I included the code to remove the origin.  One way to get the right answer would be to remove that line, but technically I think it's missing from the other end.  I think the `.index` method will return 2 meaning 2-th i.e. it has taken 3 steps

In [28]:
# for each crossed path, sum the steps it took both wires to arrive there
steps = {"path": [], "steps": []}
for path in crossed_paths:
    steps["path"].append(path)
    steps["steps"].append(positions["wire_a"].index(path) + 1
                          + positions["wire_b"].index(path) + 1)

# find the minimum number of steps it took to reach one intersection
print(f"""
The min. number of steps to reach an intersection is {min(steps["steps"])}.
""")


The min. number of steps to reach an intersection is 610.



Correct

In [29]:
example_input = """R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"""
wires = [wire.split(",") for wire in example_input.split("\n")]

relative_positions = {"wire_a": calculate_relative_coords(wires[0]),
                      "wire_b": calculate_relative_coords(wires[1])}

positions = {"wire_a": calculate_positions(relative_positions["wire_a"]),
             "wire_b": calculate_positions(relative_positions["wire_b"])}

crossed_paths = list(set(positions["wire_a"]).intersection(positions["wire_b"]))

crossed_paths.remove(ORIGIN)

# remove the origin from both wires
for wire in positions:
    positions[wire].remove(ORIGIN)
    
# for each crossed path, sum the steps it took both wires to arrive there
steps = {"path": [], "steps": []}
for path in crossed_paths:
    steps["path"].append(path)
    steps["steps"].append(positions["wire_a"].index(path) + 1
                          + positions["wire_b"].index(path) + 1)

# find the minimum number of steps it took to reach one intersection
print(f"""
The min. number of steps to reach an intersection is {min(steps["steps"])}.
""")


The min. number of steps to reach an intersection is 410.



Correct.  So now I'm going to change `part_01_solution` and `part_02_solution` so they have the updated code

Checking part 1:

In [30]:
from part_01_solution import *

[autoreload of part_01_solution failed: Traceback (most recent call last):
  File "/Users/ehoffman/.conda/envs/prework-labs/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/ehoffman/.conda/envs/prework-labs/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/Users/ehoffman/.conda/envs/prework-labs/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/Users/ehoffman/.conda/envs/prework-labs/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/ehoffman/Development/DS/advent_of_code_2019/day_03/part_01_solution.py", line 54, in <m

Mistake in my updated test code, somehow I made these all left/right instructions

In [32]:
from part_01_solution import *


There were 33 times the paths of the wires crossed.
The min. distance from one and the point of origin was 227.



Checking part 2:

In [33]:
from part_02_solution import *


The min. number of steps to reach an intersection is 20286.

